# MATH 693A Advanced Numerical Methods: Computational Optimization HW 3
### By Will McGrath

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy import interpolate

# Problem 1
### Write a Program that implements the dogleg method. Choose $B_k$ to be the exact Hessian. Apply it to solve Rosenbrock’s function: $f(x) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2$. 
### Use an initial trust region radius of 1. Set maximum trust region radius to 300. Use the initial point: $x_0 = [-1.2, 1]$ and then try another point $x_0 = [2.8, 4]$. Do the following for each of the initial points.

### Here are the parameters you should use for the Dogleg Algorithm:
### a. Use $|| \nabla{f(x_k)} || < 10^{-8}$ as the stopping criteria for your optimization algorithm.
### b. State the total number of iterations obtained in your optimization algorithm.
### c. Plot the objective function $f(x)$. On the same figure, plot the $x_k$ values at the different iterates of your optimization algorithm.
### d. Plot the size of the objective function as a function of the iteration number. Use semi-log plot.
### e. You should hand in (i) your code (ii) the first 4 and last 4 values of $x_k$ obtained from your program.
### f. Determine the minimizer of the Rosenbrock function $x^*$.


# Problem 2
### Experiment with the update rule for the trust region by changing the constants in Algorithm 4.1 in the text Numerical Optimization by Nocedal and Wright 2006. State what you experimented with and discuss your observations.

In [ ]:
def jumpingOnClouds(c):
    # Write your code here
    num_of_jumps = 0
    indx = 0
    while indx < len(c)-1:
        if indx+2 < len(c) and c[indx+2] == 0:
            indx += 2
            num_of_jumps +=1

        else:
            indx += 1
            num_of_jumps += 1

    return num_of_jumps

In [158]:
c = [0,0,1,0,0,0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0]
#c = [0,0]
jumpingOnClouds(c)

28